In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120'

In [3]:
response = requests.get(url)

In [4]:
soup = BeautifulSoup(response.text)

In [8]:
soup.find('table')

<table>
<caption>
    U.S. congressional apportionment
  </caption>
<thead>
<tr>
<th id="s">
        state
      </th>
<th id="r">
        representatives
      </th>
</tr>
</thead>
<tbody>
<tr>
<td headers="s" scope="row">
        Alabama
      </td>
<td headers="r">
        7
      </td>
</tr>
<tr>
<td headers="s" scope="row">
        Alaska
      </td>
<td headers="r">
        1
      </td>
</tr>
<tr>
<td headers="s" scope="row">
        Arizona
      </td>
<td headers="r">
        9
      </td>
</tr>
<tr>
<td headers="s" scope="row">
        Arkansas
      </td>
<td headers="r">
        4
      </td>
</tr>
<tr>
<td headers="s" scope="row">
        California
      </td>
<td headers="r">
        52
      </td>
</tr>
<tr>
<td headers="s" scope="row">
        Colorado
      </td>
<td headers="r">
        8
      </td>
</tr>
<tr>
<td headers="s" scope="row">
        Connecticut
      </td>
<td headers="r">
        5
      </td>
</tr>
<tr>
<td headers="s" scope="row">
        Delaware
 

In [16]:
# code inside read_html results in a list with one element, flatten by calling [0], df results
# drop index 50 (total row)


state_rep_num = pd.read_html(str(soup.find('table')))[0].drop(50)

In [22]:
# need list of state abbreviations

abb_url = 'https://www.yourdictionary.com/articles/state-abbreviations'

abb_res = requests.get(abb_url)

abb_soup = BeautifulSoup(abb_res.text)

In [26]:
state_abb = pd.read_html(str(abb_soup.find('table')), header = 0)[0].drop('Traditional Abbreviation', axis = 1)

In [27]:
print(state_rep_num.columns)
print(state_abb.columns)

Index(['state', 'representatives'], dtype='object')
Index(['State Name', 'USPS Abbreviation'], dtype='object')


In [63]:
# join state_rep_num to state_abb

st_rep = pd.merge(state_rep_num, state_abb, left_on = 'state', right_on = 'State Name', how = 'inner')\
    .drop('State Name', axis = 1).rename({'USPS Abbreviation': 'state_abbr'}, axis = 1)

st_rep.head(1)

,state,representatives,state_abbr
0,Alabama,7,AL


In [69]:
# example url
# https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N.

url_base = 'https://www.opensecrets.org/races/candidates?cycle=2020&id='
url_suffix = '&spec=N'

url_list = []

for r in range(len(st_rep)):
    abbr = st_rep.loc[r]['state_abbr']
    rep_num = st_rep.loc[r]['representatives']

    for d in range(1, rep_num + 1):
        url = f"{url_base}{abbr}{str(d).zfill(2)}{url_suffix}"
        url_list.append(url)

st_rep['urls']

435

In [73]:
pd.DataFrame(url_list).to_csv('../data/dist_urls.csv')